# Explore GeMAPS
Use this code to find out how the features of the GeMAPS feature set are extracted

Most of the logic for this Notebook is hidden in the class "Custom_GeMAPS"

In [ ]:
# Imports
# Python packages
import numpy as np
import matplotlib.pyplot as pltLK
import IPython.display as ipd
from opensmile import Smile, FeatureSet, FeatureLevel
import librosa as lr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy

import os
import random 

In [ ]:
audio_path = "/home/chr1s/Downloads/cv-corpus-21.0-2025-03-14/en/clips/"

# Load an audiofile
Load an audiofile into the Custom_GeMAPS class.
Its best to use a Wave file.
You can customize the samplerate.

When the file is found the class has the sample_rate (sr) and the array of samples (y).

In [ ]:
sr = 16_000 # This is the sample rate that egemaps also uses
audio_file: str = os.path.join(audio_path, random.choice(os.listdir(audio_path)))
y, sr = lr.load(audio_file, sr=sr)
print(f"Audio file: {audio_file}")
# play the audio
ipd.Audio(filename=audio_file, autoplay=True)

In [ ]:
# Plot the samples
plt.figure(figsize=(12, 4))
plt.plot(y)

plt.title("Audio Signal")
plt.xlabel("Sample Number")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
# Plot a short timeframe of the audio signal 
plt.figure(figsize=(12, 4))
length = 0.02 # lenghth of the timeframe in seconds
offset = 0.5 # offset in seconds
time_start = int(sr * offset)
time_end = time_start + int(sr * length)
plt.plot(y[time_start:time_end])

plt.title("Audio Signal")
plt.xlabel("Sample Number")
plt.ylabel("Amplitude")
plt.show()

### Remove noise
Lets remove some noise from the signal before we start working on it!

# Smile :\)

Use the [OpenSMILE](https://d-nb.info/1248485475/34) tool for feature extraction.

The list of features is a modified version of the [GeMAPS Feature Set](https://mediatum.ub.tum.de/doc/1523509/file.pdf). The functionality and meaning of those features are explored in the Notebook below.

In [ ]:
smile = Smile(
    feature_set=FeatureSet.eGeMAPSv02,
    feature_level=FeatureLevel.Functionals,
)

features = smile.process_signal(y, sr).iloc[0]
features.shape


In [ ]:
features

In [ ]:
smile.feature_names

## What?!

88 Features are still quite many, but compared to the original sr*seconds float variables a very high reduction of information.

Now we need to know what each of those 88 values mean.

## 1. Statistics!

Most of the values are means (mean) and standard deviations (std).
The basic apporach in this kind of feature set is, to measure each feature multiple times, by windowing the audio file.

But this would 1. still be a lot of data and 2. different lengths of audio files would have a different count of features.

By taking the average (mean) and standard deviation values of one feature we are left with only two values and a good rough understanding of how the property behaves most of the time.

In [ ]:
array1 = np.random.normal(loc=5, scale=1.5, size=100)
array2 = np.random.normal(loc=8, scale=0.8, size=100)
mean1, std1 = np.mean(array1), np.std(array1)
mean2, std2 = np.mean(array2), np.std(array2)

sns.swarmplot(data=[array1, array2], palette=['skyblue', 'salmon'], size=6)
plt.axhline(y=mean1, color='skyblue', linestyle='-', linewidth=2)
plt.axhline(y=mean1-std1, color='skyblue', linestyle='--', linewidth=1)
plt.axhline(y=mean1+std1, color='skyblue', linestyle='--', linewidth=1)
plt.axhline(y=mean2, color='salmon', linestyle='-', linewidth=2)
plt.axhline(y=mean2-std2, color='salmon', linestyle='--', linewidth=1)
plt.axhline(y=mean2+std2, color='salmon', linestyle='--', linewidth=1)


plt.title('Mean and Standard Deviation', fontsize=14)
plt.xticks([0, 1], ['Feature 1', 'Feature 2'], fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Percentiles

To have an even better statistical insight of the data for more important features percentile values are used.
Percentiles describe at which point X % of the data is present.
A 50% percentile is the median value.

In [ ]:
p20 = np.percentile(array1, 20)
p50 = np.percentile(array1, 50)
p80 = np.percentile(array1, 80)

plt.hist(array1, bins=80, color="skyblue")
plt.axvline(x=p20, color="salmon", linestyle="--")
plt.axvline(x=p50, color="salmon", linestyle="--")
plt.axvline(x=p80, color="salmon", linestyle="--")
plt.axvline(x=mean1, color="salmon", linestyle="-")
plt.show()

# Before Feature Extraction!

Before we can actually extract relevant features and their statistics over time we need to standardize the data.
We're going to do this to visualize how some of the features are actually calculated.

In [ ]:
# Check the data type for float 32
print(f"Data type: {y.dtype}")

In [ ]:
# Normalize the signal between -1 and 1
#y = (y - np.min(y)) / (np.max(y) - np.min(y)) * 2 - 1

In [ ]:
"""
# Apply a band pass filter
# This is to reduce noise <20Hz  and remove unusable frequencies > 8kHz
lowcut = 20.0
highcut = 8000.0

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    highcut = min(highcut, nyq * 0.99)
    
    low = lowcut / nyq
    high = highcut / nyq
    if low <= 0 or high >= 1:
        raise ValueError(f"Invalid cutoff frequencies: low={lowcut}Hz, high={highcut}Hz. "
                       f"For fs={fs}Hz, valid range is 0 < low < high < {nyq}Hz")
    b, a = scipy.signal.butter(order, [low, high], btype='band')

    y = scipy.signal.filtfilt(b, a, data)
    return y


y = butter_bandpass_filter(y, lowcut, highcut, sr)
"""

In [ ]:
# Windowing 60ms 20ms
frame_length = int(0.06 * sr)  # 60ms
hop_length = int(0.01 * sr)  # 10ms
y_windowed = lr.util.frame(y, frame_length=frame_length, hop_length=hop_length, axis=0)

print(frame_length, hop_length, y_windowed.shape)

start_plot = 6

plt.figure(figsize=(12, 4))
for i in range(start_plot, start_plot+4):
    plt.subplot(1, 4, i+1-start_plot)
    sns.lineplot(data=y_windowed[i])
    plt.axvline(x=hop_length, color="salmon", linestyle="--")
    plt.title(f'Window {i+1}')
plt.tight_layout()
plt.show()

In [ ]:
# generate a window function
gaussian_window = scipy.signal.windows.gaussian(frame_length, std=0.4*frame_length)

plt.figure(figsize=(12, 4))
plt.subplot(1, 4, 1)
sns.lineplot(data=gaussian_window)
plt.title(f'Gaussian Window')
for i in range(start_plot, start_plot+4):
    plt.subplot(1, 4, i-start_plot+1)
    sns.lineplot(data=y_windowed[i])
    plt.axvline(x=hop_length, color="salmon", linestyle="--")
    plt.title(f'Window {i+1}')
plt.tight_layout()
plt.show()

In [ ]:
# Calcualte the fourier transform
stft = lr.stft(y, hop_length=hop_length, win_length=frame_length, window=gaussian_window, center=False)

magnitude = np.abs(stft)
db_spectrogram = lr.amplitude_to_db(magnitude, ref=np.max)

single_frame = db_spectrogram[:, start_plot]

frequencies = lr.fft_frequencies(sr=sr, n_fft=single_frame.shape[0]*2-1)

min_freq = 0
min_freq_index = np.where(frequencies >= min_freq)[0][0]
max_freq = 4000
max_freq_index = np.where(frequencies <= max_freq)[0][-1]

plt.figure(figsize=(10, 6))
plt.plot(frequencies[min_freq:max_freq_index], single_frame[min_freq:max_freq_index])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.title('Single STFT Frame')
plt.grid(True)
plt.show()

In [ ]:
lr.display.specshow(
    db_spectrogram,
    sr=sr,
    hop_length=hop_length,
    x_axis="time",
    y_axis="log",
    cmap="magma",
)

# Frequency Related Parameters

Lets take a look at the frequency related parameters of the feature set.
The list of frequency related parameters is the following:

### Fundamental Frequency
|Name|Bedeutung|
|-|-|
'F0semitoneFrom27.5Hz_sma3nz_amean'|Mittelwert
'F0semitoneFrom27.5Hz_sma3nz_stddevNorm'|Standartabweichung
'F0semitoneFrom27.5Hz_sma3nz_percentile20.0'|Perzentil 20
'F0semitoneFrom27.5Hz_sma3nz_percentile50.0'|Perzentil 50 / Median
'F0semitoneFrom27.5Hz_sma3nz_percentile80.0'|Perzentil 80
'F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2'|Platz zwischen Perzentil 0-2
'F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope'|Steigung am Median
'F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope'|Steigung an der Standartabweichung
'F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope'|Senkung am Median
'F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope'|Senkung an der Standartabweichung

In [ ]:
amean = features["F0semitoneFrom27.5Hz_sma3nz_amean"]
std = features["F0semitoneFrom27.5Hz_sma3nz_stddevNorm"]
pct_20 = features["F0semitoneFrom27.5Hz_sma3nz_percentile20.0"]
pct_50 = features["F0semitoneFrom27.5Hz_sma3nz_percentile50.0"]
pct_80 = features["F0semitoneFrom27.5Hz_sma3nz_percentile80.0"]

amean, std, pct_20, pct_50, pct_80

In [ ]:
# fundamental_freq
lr.display.specshow(
    db_spectrogram,
    sr=sr,
    hop_length=hop_length,
    x_axis="time",
    y_axis="log",
    cmap="magma",
)

plt.axhline(y=amean, color="white", linestyle="--", label="F0 Mean")
plt.axhline(y=amean*4, color="white", linestyle="--", linewidth=1, label="F0 Mean * 4")
plt.axhline(y=amean+std, color="gray", linestyle="--", label="F0 Std")
plt.axhline(y=amean-std, color="gray", linestyle="--")
plt.axhline(y=pct_20, color="blue", linestyle="--", label="Percentiles")
plt.axhline(y=pct_50, color="blue", linestyle="--")
plt.axhline(y=pct_80, color="blue", linestyle="--")

plt.legend(loc="upper right")
plt.show()

### Energy / Amplitude realted Parameters

### Spectral (balance) Parameters

### Temporal Paramers

### Extended Parameter Set